<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/VIT_ALL_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentence-transformers Vs transformers Vs timm Vs facebookresearch Vs vit-pytorch

## sentence-transformers by UKPLab (To classify text)

page: https://www.libhunt.com/r/sentence-transformers

github: https://github.com/UKPLab/sentence-transformers

Installation

In [1]:
! pip -q install sentence-transformers

Text Feature Extraction

In [2]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

query = "I ate dinner"
query_vec = sbert_model.encode([query])[0]
print('Sample BERT embedding vector - length', len(query_vec))

## transformer by huggingface (ViT: To classify Image)
page: https://www.libhunt.com/r/transformers

Github: https://github.com/huggingface/transformers

Installation

In [4]:
! pip -q install transformers

In [5]:
from PIL import Image
import numpy as np
import torchvision

from transformers import ViTFeatureExtractor, ViTForImageClassification, BatchFeature
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

import torch


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=58, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=64, type=int, help='batch size')
parser.add_argument('--num_epoch', default=2, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=100, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Iter:',batch_idx,'/',len(trainloader), ' Loss:',loss.item())

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs).logits
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

class ViTFeatureExtractorTransforms:
    def __init__(self, model_name_or_path):
        feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
        transform = []

        if feature_extractor.do_resize:
            transform.append(Resize(feature_extractor.size))

        transform.append(ToTensor())

        if feature_extractor.do_normalize:
            transform.append(Normalize(feature_extractor.image_mean, feature_extractor.image_std))

        self.transform = Compose(transform)

    def __call__(self, x):
        return self.transform(x)


def main():
    seed_everything()
    model_name_or_path = 'google/vit-base-patch16-224-in21k'
    transform_vit = ViTFeatureExtractorTransforms(model_name_or_path)
    train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_vit)
    test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_vit)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    model = ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=args.num_classes)
    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
    criterion = nn.CrossEntropyLoss()

    best_epoch, best_acc = 0.0, 0
    for epoch in range(args.num_epoch):
        train(model, train_loader, criterion, optimizer)
        accuracy = test(model, test_loader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_cifar10h_vit.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))
        
main()

Files already downloaded and verified
Files already downloaded and verified


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Iter: 0 / 863  Loss: 4.627674579620361
Iter: 100 / 863  Loss: 2.7157750129699707
Iter: 200 / 863  Loss: 1.976897954940796
Iter: 300 / 863  Loss: 2.129338264465332
Iter: 400 / 863  Loss: 1.4080332517623901
Iter: 500 / 863  Loss: 1.377852201461792
Iter: 600 / 863  Loss: 1.5307177305221558
Iter: 700 / 863  Loss: 1.1656787395477295
Iter: 800 / 863  Loss: 1.4090977907180786
epoch: 0  acc: 0.6377  best epoch: 0  best acc: 0.6377
Iter: 0 / 863  Loss: 1.3454694747924805
Iter: 100 / 863  Loss: 1.5767067670822144
Iter: 200 / 863  Loss: 0.8645036220550537
Iter: 300 / 863  Loss: 0.9081160426139832
Iter: 400 / 863  Loss: 0.9121677875518799
Iter: 500 / 863  Loss: 0.9224331974983215
Iter: 600 / 863  Loss: 0.9078149199485779
Iter: 700 / 863  Loss: 0.7008175253868103
Iter: 800 / 863  Loss: 0.7572551965713501
epoch: 1  acc: 0.7246  best epoch: 1  best acc: 0.7246


# Convnext timm library

Installation

In [1]:
! pip -q install timm

     |████████████████████████████████| 509 kB 15.8 MB/s 


In [2]:
from PIL import Image
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np

from timm import create_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=48, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=64, type=int, help='batch size')
parser.add_argument('--num_epoch', default=2, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=100, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Iter:',batch_idx,'/',len(trainloader), ' Loss:',loss.item())

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total


def main():
    seed_everything()

    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
    test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    #model = create_model('vit_base_patch16_224', pretrained=True)
    model =  create_model('convnext_base', pretrained=True)
    model.head.fc = torch.nn.Linear(model.head.fc.in_features, 100)
    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
    criterion = nn.CrossEntropyLoss()

    best_epoch, best_acc = 0.0, 0
    for epoch in range(args.num_epoch):
        train(model, train_loader, criterion, optimizer)
        accuracy = test(model, test_loader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_cifar10h_vit.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))
        
main()

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


Iter: 0 / 1042  Loss: 4.6225738525390625
Iter: 100 / 1042  Loss: 1.4913959503173828
Iter: 200 / 1042  Loss: 1.5730472803115845
Iter: 300 / 1042  Loss: 1.022673487663269
Iter: 400 / 1042  Loss: 1.0205610990524292
Iter: 500 / 1042  Loss: 1.2631632089614868
Iter: 600 / 1042  Loss: 0.9487571716308594
Iter: 700 / 1042  Loss: 0.694162130355835
Iter: 800 / 1042  Loss: 1.0163886547088623
Iter: 900 / 1042  Loss: 1.1024901866912842
Iter: 1000 / 1042  Loss: 0.8531392216682434
epoch: 0  acc: 0.7615  best epoch: 0  best acc: 0.7615
Iter: 0 / 1042  Loss: 0.5325803160667419
Iter: 100 / 1042  Loss: 0.5399768948554993
Iter: 200 / 1042  Loss: 0.14133308827877045
Iter: 300 / 1042  Loss: 0.966590940952301
Iter: 400 / 1042  Loss: 0.4747079312801361
Iter: 500 / 1042  Loss: 0.7641847133636475
Iter: 600 / 1042  Loss: 0.8159897923469543
Iter: 700 / 1042  Loss: 0.9052162170410156
Iter: 800 / 1042  Loss: 0.4674842655658722
Iter: 900 / 1042  Loss: 0.6804144382476807
Iter: 1000 / 1042  Loss: 0.6426638960838318
epo

# facebookresearch

In [ ]:
from PIL import Image
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=64, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=64, type=int, help='batch size')
parser.add_argument('--num_epoch', default=10, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=100, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 300 == 0:
            print('Iter:',batch_idx,'/',len(trainloader), ' Loss:',loss.item())

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total


def main():
    seed_everything()

    preprocess = transforms.Compose([
            transforms.Resize(224),
            #transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
    test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    model = torch.hub.load('facebookresearch/deit:main',
                           'deit_tiny_patch16_224', pretrained=True)
    model.head = torch.nn.Linear(model.head.in_features, 100)
    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
    criterion = nn.CrossEntropyLoss()

    best_epoch, best_acc = 0.0, 0
    for epoch in range(args.num_epoch):
        train(model, train_loader, criterion, optimizer)
        accuracy = test(model, test_loader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_cifar10h_vit.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))
        
main()

# vit-pytorch

Installation

In [2]:
! pip -q install vit-pytorch

In [14]:
import torch
from vit_pytorch import ViT


from PIL import Image
import numpy as np
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Resize, Compose

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=12):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--batch_size', default=64, type=int, help='batch size')
parser.add_argument('--test_batch_size', default=64, type=int, help='batch size')
parser.add_argument('--num_epoch', default=2, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=100, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Iter:',batch_idx,'/',len(trainloader), ' Loss:',loss.item())

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total


def main():
    seed_everything()

    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
    test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True,num_workers=2)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False, num_workers=2)

    #model = create_model('vit_base_patch16_224', pretrained=True)
    model = ViT(
    image_size = 224,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
    )       
    model.mlp_head[1] = nn.Linear(model.mlp_head[1].in_features, 100)
    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, nesterov=False, weight_decay=0.0001)
    criterion = nn.CrossEntropyLoss()

    best_epoch, best_acc = 0.0, 0
    for epoch in range(args.num_epoch):
        train(model, train_loader, criterion, optimizer)
        accuracy = test(model, test_loader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_cifar10h_vit.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))
        
main()



Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: ignored